In [167]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta
from matplotlib.pyplot import cm
from scipy.stats import pearsonr,spearmanr

from minepy import MINE
from minepy import pstats, cstats

%matplotlib inline


In [178]:
x = np.linspace(0, 1, 1000)
y = np.sin(10 * np.pi * x) + x**2 #+ np.random.randint(2, size=1000)
y = x**2
mine = MINE(alpha=0.9, c=15, est="mic_e")
mine.compute_score(x, y)

print ("Without noise:")
print(mine.mic())

/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


Without noise:
1.0000000000000033


In [223]:
path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Afghanistan.csv'
path = '~/data/ACLED/country-jul23/2018-01-01-2021-07-23-Brazil.csv'
# path = '~/data/ACLED/country-jul23/2015-01-01-2021-07-23-Yemen.csv'
# path = '~/data/ACLED/country-jul23/2016-01-01-2021-07-23-Turkey.csv'
# path = '~/data/ACLED/country-jul23/2020-01-01-2021-07-23-United_States.csv'
# path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Syria.csv'

filename = path.split('/')[-1]
print('path',path,'filename',filename)
start_year = int(filename.split('-')[0])
start_month = int(filename.split('-')[1])
start_day = int(filename.split('-')[2])
end_year = int(filename.split('-')[3])
end_month = int(filename.split('-')[4])
end_day = int(filename.split('-')[5])

df = pd.read_csv(path,sep=';')
df = df.drop_duplicates(subset=['data_id'], keep='first')
df['event_date'] = pd.to_datetime(df['event_date'])
print(df.columns)

df.sort_values(by=['event_date'],inplace=True ) 
print(df.value_counts('year'))
# print(df['admin1'].unique().shape)
# df.value_counts('admin1')
# df.value_counts('sub_event_type')

path ~/data/ACLED/country-jul23/2018-01-01-2021-07-23-Brazil.csv filename 2018-01-01-2021-07-23-Brazil.csv
Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')
year
2018    16498
2019    11252
2020    10078
2021     6371
dtype: int64


In [224]:
event_type_column = 'sub_event_type'
# event_type_column = 'event_type'
delta_value = 1
if delta_value == 1:
    level = 'day'
elif delta_value == 7:
    level = 'week'
elif delta_value == 14:
    level = 'biweek'
elif delta_value == 30:
    level = 'month'
subevents = df[event_type_column].unique()
print(len(subevents),subevents)
subevent_count_dict = {}
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
n_days = 0
last_date = start_date - delta
while start_date <= end_date:
#     print('last_date',last_date,'start_date',start_date )
    last_date = start_date
    start_date += delta
    n_days += 1
print('n_days =',n_days)
# print('n_days =',len(df['event_date'].unique()))
for v in subevents:
    subevent_count_dict[v] = np.array([0 for i in range(n_days)])


18 ['Remote explosive/landmine/IED' 'Armed clash' 'Mob violence'
 'Peaceful protest' 'Attack' 'Violent demonstration'
 'Disrupted weapons use' 'Looting/property destruction'
 'Protest with intervention' 'Arrests' 'Abduction/forced disappearance'
 'Non-violent transfer of territory' 'Sexual violence'
 'Change to group/activity' 'Other' 'Excessive force against protesters'
 'Grenade' 'Chemical weapon']
n_days = 1300


In [225]:
# for loop day.... save count of each subevent.
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
day_i = 0
last_date = start_date - delta
# print('last_date',last_date,'start_date',start_date,'end_date',end_date)

while start_date <= end_date:
    last_date_str = last_date.strftime("%Y-%m-%d") #("%d %B %Y")
    date_str = start_date.strftime("%Y-%m-%d")
#     print('last_date_str',last_date_str,' --- date_str',date_str)
    df_day = df.loc[(df['event_date'] > last_date_str) & (df['event_date'] <= date_str)]
    if day_i%300==0:
        print('#',len(df_day),len(df))
    df_count = df_day[event_type_column].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for i,row in df_count.iterrows():
        subevent_count_dict[row['unique_values']][day_i] = row['counts']
    last_date = start_date
    start_date += delta
    day_i += 1
print('day_i =',day_i)

# 31 44199
# 17 44199
# 33 44199
# 31 44199
# 36 44199
day_i = 1300


In [226]:
subevent_count_dict['Armed clash']
subevent_count_dict['Other']

array([0, 0, 0, ..., 0, 0, 0])

In [227]:
# get outcome variable 
protests_count = subevent_count_dict['Protest with intervention'] + subevent_count_dict['Peaceful protest'] + subevent_count_dict['Excessive force against protesters']
protests_count = np.where(protests_count < 1, 0, 1)
print('mean',protests_count.mean(),protests_count.shape)

# get treatment variables
def get_avg(x,day=8):
    r = []
    for i in range(0,len(x),day):
        avg = x[i:i+day].mean()
        r += [avg for i in range(day)]
    r = np.array(r)
#     print(r)
    return r
# protests_count[16:].shape
         

mean 0.9646153846153847 (1300,)


In [228]:
def moving_average(a, n=3) :
    padding = []
    for i in range(n-1):
        padding.append(a[:i+1].mean())
    padding = np.array(padding)
#     print('padding',padding,padding.shape)
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return np.concatenate((padding, ret[n - 1:] / n),0)
event_set_protest = ['Protest with intervention','Excessive force against protesters','Peaceful protest']
subevent_count_dict['Protests'] = subevent_count_dict['Protest with intervention'] + subevent_count_dict['Peaceful protest'] + subevent_count_dict['Excessive force against protesters']
bi_trend_dict = {}
ma_day = 14
subevents = list(subevents) +['Protests']
for i in range(len(subevents)):
    if subevents[i] in event_set_protest:
        continue
    count_data = subevent_count_dict[subevents[i]]
#     count_data = np.array([1,1,1,2,4,4,5,6,8,8,2,12,0,0,1,0,2,0,24,10,5,6,8,8,2,10,5,6,8,8,2,10,5,6,8,8,2])
    ma = moving_average(count_data,ma_day)
    cur_trend = ma[ma_day*2-1:]
    prev_trend = ma[ma_day-1:ma_day-1+len(cur_trend)]
#     print(prev_trend,'prev')
#     print(cur_trend,'curr')
    bi_trend = np.where((cur_trend-prev_trend)>0,1,0)
    bi_trend_dict[subevents[i]] = bi_trend
#     break
print(bi_trend[:],bi_trend.shape)



[1 1 1 ... 0 0 0] (1273,)


In [229]:
bi_trend_dict

{'Remote explosive/landmine/IED': array([0, 0, 0, ..., 0, 0, 0]),
 'Armed clash': array([0, 0, 0, ..., 1, 0, 0]),
 'Mob violence': array([1, 1, 1, ..., 0, 0, 0]),
 'Attack': array([0, 1, 1, ..., 1, 0, 0]),
 'Violent demonstration': array([1, 1, 1, ..., 0, 0, 1]),
 'Disrupted weapons use': array([0, 0, 0, ..., 0, 0, 0]),
 'Looting/property destruction': array([1, 1, 1, ..., 0, 0, 0]),
 'Arrests': array([0, 0, 0, ..., 0, 0, 0]),
 'Abduction/forced disappearance': array([1, 1, 1, ..., 0, 1, 1]),
 'Non-violent transfer of territory': array([0, 0, 0, ..., 0, 0, 0]),
 'Sexual violence': array([0, 0, 0, ..., 0, 0, 0]),
 'Change to group/activity': array([0, 0, 0, ..., 0, 0, 0]),
 'Other': array([0, 0, 0, ..., 0, 0, 0]),
 'Grenade': array([0, 0, 0, ..., 0, 0, 0]),
 'Chemical weapon': array([0, 0, 0, ..., 0, 0, 0]),
 'Protests': array([1, 1, 1, ..., 0, 0, 0])}

In [230]:
# get association
data2 = protests_count[ma_day*2:]
print(data2.shape)

# x = np.linspace(0, 1, 1000)
# y = np.sin(10 * np.pi * x) + x



for i in range(len(subevents)):
    if subevents[i] in event_set_protest:
        continue
    data1 = bi_trend_dict[subevents[i]][:-1]
    corr, p = pearsonr(data1, data2)
    print(subevents[i],'pearsonr',corr*100,p,(p<0.05))
    corr, p = spearmanr(data1, data2)
    print(subevents[i],'spearmanr',corr*100,p,(p<0.05))
    
    mine = MINE(alpha=0.6, c=15, est="mic_e")
    mine.compute_score(data1, data2)
#     mic_p, tic_p = cstats(data1, data1, alpha=0.6, c=15, est="mic_e")
    print(mine.mic()*100)

(1272,)
Remote explosive/landmine/IED pearsonr -3.8891848834379292 0.16567278107477704 False
Remote explosive/landmine/IED spearmanr -3.889184883437926 0.16567278107443204 False
0.10695044910324386
Armed clash pearsonr 0.2083504225953995 0.9408230548275984 False
Armed clash spearmanr 0.20835042259540762 0.9408230548254839 False
0.0003131179477555733
Mob violence pearsonr 2.1139701897786094 0.4512740426575188 False
Mob violence spearmanr 2.113970189778611 0.45127404265652593 False
0.03257790278347102
Attack pearsonr -0.4465623653537555 0.8735812922724857 False
Attack spearmanr -0.4465623653537522 0.8735812922706838 False
0.001438110764874733
Violent demonstration pearsonr -1.1581766719167097 0.6798464229978743 False
Violent demonstration spearmanr -1.1581766719167033 0.6798464229963883 False
0.00967580792509868
Disrupted weapons use pearsonr 0.9037358372717113 0.7474465885083447 False
Disrupted weapons use spearmanr 0.9037358372717118 0.7474465885068602 False
0.005989213416879837
Lootin

/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [118]:
data2.mean()

0.2565668906536347

In [115]:
# # check moving average
# def movingaverage(interval, window_size):
#     window = np.ones(int(window_size))/float(window_size)
#     return np.convolve(interval, window, 'same')

# t = np.arange(1, day_i+1, 1) 
# color=cm.rainbow(np.linspace(0,1,len(subevents)))

# fig, axs = plt.subplots(1, 1,figsize=(16, 8))
# event_set_protest = ['Protest with intervention','Excessive force against protesters','Peaceful protest']
# event_set = ['Battles', 'Explosions/Remote violence', 'Strategic developments','Violence against civilians', 'Protests', 'Riots']
# # event_set = ['Battles', 'Explosions/Remote violence', 'Strategic developments','Violence against civilians', 'Riots']
# event_set = [  'Strategic developments','Violence against civilians', 'Protests', 'Riots']

# # event_set = ['Violence against civilians', 'Protests', 'Riots']
# # event_set = [  'Explosions/Remote violence']
# # color2 = ['#080808','#333030','#6e6a6a']
# for i in range(len(subevents)):
# #     print(subevents[i])
#     if subevents[i] in event_set_protest:
#         ma_day = 8

#         count_data = subevent_count_dict[subevents[i]][:88]
#         axs.plot(t[:88], count_data, 's',label=subevents[i])
        
#         ma = movingaverage(count_data,ma_day)
#         cur_label = subevents[i]+'_'+str(ma_day)+'MovAvg'
#         if ma_day == 1:
#             cur_label = subevents[i]
         
#         axs.plot(t[:88], ma, 'o-',label=cur_label,color=color[i])
# #         avg_data = get_avg(count_data,day=8)
# #         axs.plot(t[:88],avg_data, color=color[i],  linestyle='--',label=subevents[i]+'_'+'Avg_of_8_steps')
# #         axs.axhline(y=count_data.mean(), color=color[i],  linestyle='--')
         

# axs.set_xlabel('week',fontsize=16)
# axs.set_ylabel('# events',fontsize=16)
# axs.grid(True)
# axs.legend(prop={'size': 16},ncol=2)
# # cxy, f = axs[1].cohere(s1, s2, 256, 1. / dt)
# # axs[1].set_ylabel('coherence')
# axs.margins(0.01)
# axs.set_title('Afghanistan day-level')
# axs.set_title('Syria week-level eg')
# # axs.set_title('Yemen week-level')

# fig.tight_layout()
# plt.show()
# # fig.savefig("Syria_week_main_eg.pdf", bbox_inches='tight')
